In [1]:
import numpy as np
import optimizer
import sinkhorn_ops
import matplotlib.pyplot as plt
from scipy.misc import imresize
from scipy.optimize import linear_sum_assignment
%matplotlib inline
from time import time
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import numpy as np

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
data_train = input_data.read_data_sets('/tmp/', one_hot=True).train 
data_test = input_data.read_data_sets('/tmp/', one_hot=True).test 


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/t10k-images-idx3-ubyte.gz
Extracting /tmp/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Extracting /tmp/train-images-idx3-ubyte.gz
Extracting /tmp/train-labels-idx1-ubyte.gz
Extracting /tmp/t10k-images-idx3-ubyte.gz
Extracting /tmp/t10k-labels-idx1-ubyte.gz


In [2]:
from scipy.special import logsumexp
def sinkhorn(log_alpha, n_iters=20):

    for n in range(n_iters):
        log_alpha -= logsumexp(log_alpha, axis=1, keepdims=True)
        log_alpha -= logsumexp(log_alpha, axis=0, keepdims=True)
        if n%10000==1:
            print(n)
    log_alpha -= logsumexp(log_alpha, axis=1, keepdims=True)
    return np.exp(log_alpha)

def batch_split(batch, n_squares_side, n_channels=1):
    if(n_channels ==1):
        #side = int(np.sqrt(batch.shape[1]))
        side =batch.shape[1]
    else:
        side = batch.shape[1]
    batch_size = batch.shape[0]
    n_squares = n_squares_side ** 2
    
    batch = np.reshape(batch, [-1, side, side, n_channels])
    batch = np.reshape(batch, [batch_size, n_squares_side, side/n_squares_side, side, n_channels])
    batch = np.transpose(batch, [0, 2, 1, 3, 4])
    batch = np.reshape(batch, [batch_size, side/n_squares_side, n_squares, side/n_squares_side, n_channels])
    batch = np.transpose(batch, [0, 2, 1, 3, 4])
    return batch

def stack_batch_split(batch):
    return np.reshape(batch, [batch.shape[0]*batch.shape[1], batch.shape[2], batch.shape[3], batch.shape[4]])


def unflatten_batch(batch, n_channels=1):
    side_square = int(np.sqrt(batch.shape[2]/n_channels))
    return np.reshape(batch, [batch.shape[0], batch.shape[1], side_square, side_square, n_channels])

def join_batch_split(batch):
    batch_size = batch.shape[0]
    n_squares = batch.shape[1]
    side_quare = batch.shape[2]
    n_channels = batch.shape[4]
    n_squares_side = int(np.sqrt(n_squares))
    batch = np.transpose(batch, [0, 1, 3, 2, 4])
    batch = np.reshape(batch, [batch_size, n_squares_side, side_square*n_squares_side, side_square, n_channels])
    batch = np.transpose(batch, [0,1, 3,2,4])
    batch = np.reshape(batch, [batch_size, 1, side_square*n_squares_side, side_square*n_squares_side, n_channels])
    batch = np.reshape(batch, [batch_size, side_square*n_squares_side, side_square*n_squares_side, n_channels])
    return batch

def resized_dims(n_squares_side):
    if(n_squares_side==2):
        side = 28
        side_square = 14
    if(n_squares_side==3):
        side = 27
        side_square = 9
    if(n_squares_side==4):
        side = 28
        side_square = 7
    if(n_squares_side==5):
        side = 30
        side_square = 6
    if(n_squares_side==6):
        side = 30
        side_square = 5
    if(n_squares_side==7):
        side = 28
        side_square = 4
    if(n_squares_side==8):
        side = 32
        side_square = 4
    if(n_squares_side==9):
        side = 27
        side_square = 3
    if(n_squares_side==16):
        side = 80
        side_square = 5
    if(n_squares_side==14):
        side = 196
        side_square = 14
    if(n_squares_side==18):
        side = 8*18
        side_square = 8
    if(n_squares_side==20):
        side = 7*20
        side_square = 7
    if(n_squares_side==25):
        side = 25*6
        side_square = 6
    if(n_squares_side==30):
        side = 30*5
        side_square = 5
    if (n_squares_side == 37):
        side = 37 * 4
        side_square = 4
    if (n_squares_side == 45):
        side = 45 * 3
        side_square = 3
    return side, side_square

def resize_batch_color(batch, side_new, n_channels):
    batch_new = np.zeros((batch.shape[0], side_new, side_new, n_channels))
    side = int(np.sqrt(batch.shape[1]))
    for i in range(batch.shape[0]):
        for c in range(n_channels):
            a = imresize(batch[i,:,:,c], [side_new, side_new])
            
            a = a/255.0
            batch_new[i,:,:,c] =a
    return batch_new


def soft_to_hard(soft_perm):
    
    a,b = linear_sum_assignment(-soft_perm)
    ma = np.zeros((np.shape(soft_perm)))
    for i in range(soft_perm.shape[0]):
        ma[i, b[i]] = 1
    return ma


def load_image( infilename ) :
    img = Image.open( infilename )
    img.load()
    data = np.asarray( img, dtype="int32" )
    return data


In [3]:
# Define model params
batch_size = 20
n_iter_sinkhorn2 = 10
n_iter_sinkhorn = 200

temp = 0.05
temp2 =0.001

#mnist data
n_squares_side = 16
lr = 0.0001
n_channels = 1
rfield_size = 3
stride = 3
n_units = 5
keep_prob = 1.0
side_real = 28
opt = 'adam'
samples_per_num = 1
n_squares = n_squares_side **2
side, side_square = resized_dims(n_squares_side)
n_dim = int(side_square*side_square*n_channels)
print(n_dim)
n_dim_z = 128
n_dim_z2 = 64
n_dim_zp = 64
noise_factor = 0
np.random.seed(2)



25


In [4]:
#Now we define the main TF variables


real_split = tf.placeholder(tf.float32,[None, n_squares, side_square, side_square, n_channels])
real_split_tiled = tf.tile(real_split, [samples_per_num, 1, 1, 1, 1])
real_split_tiled = tf.reshape(real_split_tiled, [-1, n_squares, side_square ** 2* n_channels])

real_entire = tf.placeholder(tf.float32,[None, side_real, side_real, n_channels])
real_entire_tiled = tf.tile(real_entire,[samples_per_num,1,1,1])

global_step = tf.Variable(0, trainable=False)
stack_im = tf.placeholder(tf.float32,[None, side_square, side_square, 1])



Instructions for updating:
Colocations handled automatically by placer.


In [5]:

def create_z():
    #create the matrix of log_alpha, that will later will converted into a soft permutation
    #this relies on some NN processing (convolutional), see below
    fc = tf.contrib.layers.fully_connected
    flatten = tf.contrib.layers.flatten
    dropout = tf.contrib.layers.dropout
    def conv(input_image, kernel_shape, bias_shape):
        weights = tf.get_variable("weights", kernel_shape,
                                 initializer = tf.random_normal_initializer())
        biases = tf.get_variable("biases", bias_shape, 
                                initializer=tf.constant_initializer(-1))
        convolutional = tf.nn.conv2d(input_image, weights, 
                                     strides = [1, 1, 1, 1],
                                     padding="SAME")
        out_relu = tf.nn.relu(convolutional + biases)
        out_maxpool = tf.nn.max_pool(out_relu, 
                                    ksize=[1, stride, stride, 1],
                                   strides=[1, stride, stride, 1],
                                   padding="SAME")
        return out_maxpool
    def conv_and_fc(input_image, scope, n_dim_z):
        with tf.variable_scope(scope):
            conv_output = conv(input_image, [rfield_size, rfield_size, n_channels, n_units], [n_units])
        fully_connected_output_mean = dropout(tf.cast(fc(flatten(conv_output), n_dim_z, activation_fn = None), tf.float32),
                                        keep_prob)
        fully_connected_output_log_var = dropout(tf.cast(fc(flatten(conv_output), n_dim_z, activation_fn = None), tf.float32),
                                        keep_prob)
      
        return fully_connected_output_mean, fully_connected_output_log_var
    
    #with tf.variable_scope("model_params"):
    z_mean, z_log_var = conv_and_fc(real_entire_tiled,'n1', n_dim_z)
    z_mean_pieces, z_log_var_pieces = conv_and_fc(real_entire_tiled,'n2',n_dim_zp)
    #z_mean_pieces, z_log_var_pieces = conv_and_fc(stack_im,'n2',n_dim_zp)
    z_mean_pieces = tf.reshape(tf.tile(z_mean_pieces, [1, n_squares]), [batch_size*n_squares,n_dim_zp])
    #z_mean_pieces = tf.reshape(z_mean_pieces, [1, n_squares]), [batch_size*n_squares,n_dim_zp])
    
    z_log_var_pieces = tf.reshape(tf.tile(z_log_var_pieces, [1, n_squares]), [batch_size*n_squares,n_dim_zp])
    z_pieces = z_mean_pieces + tf.random.normal([batch_size*n_squares, n_dim_zp]) *tf.exp(z_log_var_pieces/2)
    aa = fc(z_pieces,1, activation_fn = None)
    
    z_pieces_fc = tf.contrib.distributions.Bernoulli(logits=aa/temp).mean()
    scrambled_split_tiled = tf.tile(tf.reshape(z_pieces_fc, [batch_size, n_squares, 1]), [1,1, n_channels*side_square**2])
    
    z  = z_mean + tf.random.normal([batch_size, n_dim_z]) *tf.exp(z_log_var/2)
    z1 = tf.reshape(fc(z, n_squares*n_dim_z2, activation_fn = None), [-1, n_squares, n_dim_z2])
    z2 = tf.reshape(fc(flatten(z_pieces), n_dim_z2, activation_fn = None), [-1,  n_dim_z2,  n_squares])
    cost = tf.nn.sigmoid(tf.matmul(z1,z2))
   
    return z,z_pieces, z_mean, z_log_var, z_mean_pieces, z_log_var_pieces, cost, scrambled_split_tiled, aa,z_pieces_fc

In [6]:


#ordered_inf = tf.matmul(inv_soft_perms_flat, scrambled_split_tiled)

z,z_pieces, z_mean, z_log_var, z_mean_pieces, z_log_var_pieces, cost, scrambled_split_tiled,aa,z_pieces_fc = create_z()
(soft_perms_inf2, _) = sinkhorn_ops.gumbel_sinkhorn(-cost, temp2, samples_per_num, noise_factor, n_iter_sinkhorn2, squeeze=False)
inv_soft_perms2 = tf.transpose(soft_perms_inf2, [0, 1, 3, 2])
inv_soft_perms2_flat = tf.reshape(tf.transpose(inv_soft_perms2, [1, 0, 2, 3]), [-1, n_squares, n_squares])

ordered_inf2 = tf.matmul(inv_soft_perms2_flat, scrambled_split_tiled)
 
rec_loss1 =tf.reduce_mean(tf.square(ordered_inf2 - real_split_tiled))
rec_loss =-tf.reduce_mean(tf.reduce_mean(real_split_tiled*tf.log(1e-5+ordered_inf2) +(1-real_split_tiled)*tf.log(1e-5+1-ordered_inf2),axis=0))*side_real**2

KL1 = 0.5 *tf.reduce_mean(tf.reduce_sum(1+z_log_var -tf.square(z_mean) -tf.exp(z_log_var), axis=1))
KL2 = 0.5 *tf.reduce_mean(tf.reduce_sum(1+z_log_var_pieces -tf.square(z_mean_pieces) -tf.exp(z_log_var_pieces), axis=1))
KL = KL1+KL2*n_squares
l2s_diff = rec_loss - KL

#l2s_diff = tf.reduce_mean(tf.square(ordered_inf2 - real_split_tiled))

opt = optimizer.set_optimizer(opt, lr, opt_eps=1e-8)
train_op = tf.contrib.training.create_train_op(l2s_diff, opt, global_step=global_step)


Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
Use tf.cast instead.


In [ ]:

init_op=tf.initialize_all_variables()
sess=tf.InteractiveSession()
sess.run(init_op)
new = tf.trainable_variables()
print(new)
n_iter = 20000
for i in range(n_iter):
    if i<n_iter-1:
        ims,_=data_train.next_batch(batch_size)
    else:
        print('hola')
        ims,_=data_test.next_batch(batch_size)        
    ims = np.expand_dims(np.reshape(ims, [-1, 28,28]),axis=3)
    ims[ims>0.5]=1
    ims[ims<0.5]=0
    np_x = resize_batch_color(ims, side, n_channels)

    real_images_split = batch_split(np_x, n_squares_side, n_channels)
    stack_real_images_split = stack_batch_split(real_images_split)
    [o2,loss, rec_loss_np,KL_np,rec_loss1_np,_, sp2,cos,z_mean_pieces_np, z_pieces_fc_np,z_log_var_pieces_np,aa_np] = sess.run([ordered_inf2, l2s_diff,rec_loss,KL,rec_loss1,train_op, soft_perms_inf2,cost,z_mean_pieces,z_pieces_fc,z_log_var_pieces,aa],{real_split:real_images_split,real_entire:ims, stack_im:stack_real_images_split})
    if(i%10==0):
        print([i, loss, rec_loss_np, KL_np, rec_loss1_np])
    if i==n_iter-1:
        ogen = []
        sgen =[]
        for k in range(5):
            [oo2,ssp2,scrambled_pieces_split_np] = sess.run([ordered_inf2,soft_perms_inf2, scrambled_split_tiled],{z:np.random.normal(0,1,(batch_size, n_dim_z)),
                                                                 z_pieces:np.random.normal(0,1,(n_squares*batch_size, n_dim_zp))})
            ogen.append(oo2)
            sgen.append(ssp2)

   


Instructions for updating:
Use `tf.global_variables_initializer` instead.
[<tf.Variable 'n1/weights:0' shape=(3, 3, 1, 5) dtype=float32_ref>, <tf.Variable 'n1/biases:0' shape=(5,) dtype=float32_ref>, <tf.Variable 'fully_connected/weights:0' shape=(500, 128) dtype=float32_ref>, <tf.Variable 'fully_connected/biases:0' shape=(128,) dtype=float32_ref>, <tf.Variable 'fully_connected_1/weights:0' shape=(500, 128) dtype=float32_ref>, <tf.Variable 'fully_connected_1/biases:0' shape=(128,) dtype=float32_ref>, <tf.Variable 'n2/weights:0' shape=(3, 3, 1, 5) dtype=float32_ref>, <tf.Variable 'n2/biases:0' shape=(5,) dtype=float32_ref>, <tf.Variable 'fully_connected_2/weights:0' shape=(500, 64) dtype=float32_ref>, <tf.Variable 'fully_connected_2/biases:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'fully_connected_3/weights:0' shape=(500, 64) dtype=float32_ref>, <tf.Variable 'fully_connected_3/biases:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'fully_connected_4/weights:0' shape=(64, 1) dtype=

/home/gomena/.local/lib/python2.7/site-packages/ipykernel_launcher.py:105: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


[0, 7881.3604, 2509.5464, -5371.814, 0.4630875]
[10, 8013.4688, 1935.9899, -6077.479, 0.41928402]
[20, 6805.7393, 1423.7576, -5381.9814, 0.32838586]
[30, 5805.8027, 1118.9397, -4686.863, 0.26443776]
[40, 4963.474, 996.4066, -3967.0674, 0.23770368]
[50, 5250.6094, 896.0764, -4354.5327, 0.22949365]
[60, 4702.371, 993.09766, -3709.2734, 0.23500659]
[70, 4697.8843, 754.4954, -3943.3887, 0.19377993]
[80, 4635.8286, 830.5722, -3805.2566, 0.2241469]
[90, 4177.354, 811.2455, -3366.1086, 0.2153083]
[100, 3962.026, 714.9699, -3247.056, 0.193195]
[110, 4051.6328, 649.41626, -3402.2166, 0.2065069]
[120, 3867.7812, 743.1234, -3124.6577, 0.21795608]
[130, 3736.6328, 634.0612, -3102.5715, 0.19889963]
[140, 3486.1592, 691.71716, -2794.442, 0.19757703]
[150, 3733.8813, 588.57214, -3145.309, 0.1857081]
[160, 3389.1763, 585.02435, -2804.1519, 0.18182799]
[170, 3450.9587, 560.8496, -2890.1091, 0.16697583]
[180, 3291.4717, 528.1525, -2763.319, 0.16195545]
[190, 3487.0466, 502.9966, -2984.05, 0.17055322]
[2

[1610, 888.1219, 362.62085, -525.50104, 0.123314515]
[1620, 831.9199, 366.88437, -465.03552, 0.12368204]
[1630, 888.27954, 389.75052, -498.529, 0.1367184]
[1640, 926.36145, 381.66544, -544.69604, 0.13063876]
[1650, 837.5857, 350.33932, -487.2464, 0.11417099]
[1660, 885.018, 351.28973, -533.7283, 0.11547961]
[1670, 795.9894, 344.04556, -451.9438, 0.11287698]
[1680, 849.4873, 372.1266, -477.36072, 0.12719096]
[1690, 853.2571, 377.61688, -475.64017, 0.12708515]
[1700, 827.9054, 371.06818, -456.83722, 0.12718892]
[1710, 830.8812, 392.74213, -438.1391, 0.13808855]
[1720, 801.41486, 367.4532, -433.96167, 0.12481227]
[1730, 850.39465, 377.91254, -472.4821, 0.12987606]
[1740, 883.22656, 391.5324, -491.69418, 0.13595264]
[1750, 835.34143, 381.93878, -453.40262, 0.13150519]
[1760, 789.1139, 387.60312, -401.51077, 0.1343494]
[1770, 820.9011, 368.37863, -452.52252, 0.12506102]
[1780, 834.931, 370.65952, -464.2715, 0.12463205]
[1790, 850.6433, 360.52338, -490.11993, 0.11892129]
[1800, 826.3794, 391

[3190, 494.0024, 338.91443, -155.08798, 0.11052228]
[3200, 507.03485, 354.57364, -152.46121, 0.1204359]
[3210, 503.4028, 360.5904, -142.81241, 0.123259164]
[3220, 519.63885, 351.57376, -168.0651, 0.117481396]
[3230, 535.98706, 359.788, -176.1991, 0.12147403]
[3240, 527.7177, 355.60458, -172.11313, 0.11851778]
[3250, 529.0893, 359.1514, -169.93791, 0.11800199]
[3260, 547.92175, 368.5757, -179.34604, 0.12428863]
[3270, 519.6932, 368.3266, -151.36658, 0.1258004]
[3280, 512.0259, 358.34946, -153.67645, 0.122278556]
[3290, 514.4384, 346.568, -167.87044, 0.11287331]
[3300, 547.0603, 374.03452, -173.02582, 0.12842622]
[3310, 504.13745, 352.72836, -151.40909, 0.11817582]
[3320, 499.83676, 346.58673, -153.25003, 0.114438064]
[3330, 510.05884, 355.72916, -154.32967, 0.12108751]
[3340, 512.63135, 349.09122, -163.54016, 0.11458384]
[3350, 514.2158, 359.08365, -155.13217, 0.12050684]
[3360, 515.00464, 357.95456, -157.05005, 0.119109064]
[3370, 501.41736, 335.78632, -165.63104, 0.1089059]
[3380, 495

...

[7920, 341.11655, 323.69604, -17.42051, 0.10173669]
[7930, 299.76666, 285.34048, -14.426186, 0.08498476]
[7940, 341.2968, 323.50403, -17.792782, 0.10361578]
[7950, 315.99866, 299.55762, -16.441032, 0.092003554]
[7960, 341.69678, 325.48782, -16.208961, 0.10380661]
[7970, 323.69528, 308.17188, -15.523396, 0.096120395]
[7980, 312.84848, 295.62247, -17.22601, 0.087378964]
[7990, 337.4377, 319.53842, -17.899305, 0.0999767]
[8000, 324.00137, 308.28693, -15.714434, 0.09631928]
[8010, 313.93747, 296.34128, -17.596197, 0.0896019]
[8020, 334.69107, 318.95395, -15.737118, 0.100315824]
[8030, 337.42535, 321.67975, -15.745602, 0.09981189]
[8040, 318.50696, 301.53992, -16.967035, 0.09402933]
[8050, 328.7882, 312.42267, -16.365543, 0.09795924]
[8060, 339.9877, 322.69446, -17.293238, 0.102348775]
[8070, 317.68845, 301.0355, -16.652946, 0.0931676]
[8080, 320.31036, 306.55612, -13.754228, 0.0948141]
[8090, 318.26883, 301.47998, -16.788855, 0.09161389]
[8100, 329.67996, 313.6919, -15.988073, 0.09666821]


[9480, 300.27692, 287.7899, -12.48704, 0.08654115]
[9490, 323.41483, 311.33795, -12.076868, 0.09813318]
[9500, 343.19443, 329.42828, -13.766137, 0.104280435]
[9510, 327.49127, 313.91983, -13.571439, 0.09824127]
[9520, 324.28812, 311.03473, -13.253395, 0.09687769]
[9530, 313.31207, 301.44385, -11.868224, 0.09219707]
[9540, 310.5072, 298.30682, -12.200375, 0.08945107]
[9550, 332.7472, 319.79938, -12.947826, 0.10048697]
[9560, 308.6547, 295.74432, -12.91037, 0.09085121]
[9570, 312.5768, 299.86972, -12.707095, 0.09295632]
[9580, 337.39746, 323.0432, -14.3542385, 0.10215163]
[9590, 332.57114, 321.24292, -11.328205, 0.1026513]
[9600, 329.41455, 317.59048, -11.824052, 0.09948984]
[9610, 322.1112, 309.33505, -12.776142, 0.09725798]
[9620, 311.1839, 298.9821, -12.201803, 0.09174075]
[9630, 319.5903, 307.98578, -11.604528, 0.09554944]
[9640, 294.17117, 282.4949, -11.676261, 0.083874695]
[9650, 350.87415, 337.969, -12.905159, 0.108617224]
[9660, 331.3968, 318.95212, -12.4446745, 0.10087661]
[9670

[11030, 308.36716, 297.54654, -10.820624, 0.09162086]
[11040, 280.31516, 270.49698, -9.818169, 0.078629956]
[11050, 306.4968, 296.111, -10.385818, 0.090741836]
[11060, 307.23254, 297.04022, -10.19233, 0.09103891]
[11070, 305.02417, 294.59048, -10.433683, 0.087951094]
[11080, 307.3569, 296.4114, -10.945481, 0.089342706]
[11090, 295.58447, 285.25967, -10.324796, 0.08635425]
[11100, 303.7197, 292.9703, -10.749393, 0.087326124]
[11110, 341.2984, 330.56787, -10.730521, 0.10384677]
[11120, 302.3834, 292.66675, -9.716649, 0.08978571]
[11130, 297.3464, 287.703, -9.643403, 0.086820744]
[11140, 321.91016, 311.27548, -10.63466, 0.096687905]
[11150, 314.37598, 303.23373, -11.142233, 0.09457799]
[11160, 318.0021, 307.67676, -10.325351, 0.096148096]
[11170, 314.8511, 304.88367, -9.967442, 0.09426281]
[11180, 356.5695, 344.6599, -11.9095745, 0.11132019]
[11190, 316.0376, 305.5519, -10.4857, 0.09579088]
[11200, 335.61853, 325.52078, -10.097733, 0.10383203]
[11210, 317.861, 308.74994, -9.111067, 0.0948

[12590, 311.22888, 302.7981, -8.430773, 0.09432762]
[12600, 307.85324, 299.45383, -8.399403, 0.09235642]
[12610, 299.3795, 290.817, -8.562488, 0.08743798]
[12620, 315.1078, 306.17828, -8.929514, 0.09383187]
[12630, 304.99515, 296.04172, -8.953434, 0.09068582]
[12640, 311.99655, 302.6981, -9.298448, 0.093450904]
[12650, 328.5359, 318.87552, -9.6603775, 0.100965396]
[12660, 312.53232, 302.205, -10.327321, 0.09340589]
[12670, 346.38037, 336.3439, -10.036475, 0.10923601]
[12680, 317.9276, 308.69366, -9.23396, 0.09624576]
[12690, 311.62875, 301.63626, -9.992478, 0.092564456]
[12700, 309.41168, 300.30698, -9.104716, 0.091231026]
[12710, 326.32166, 317.49716, -8.824492, 0.09998581]
[12720, 286.68677, 277.52237, -9.164412, 0.08221896]
[12730, 298.63306, 289.75323, -8.879819, 0.086477295]
[12740, 300.2718, 291.65366, -8.618135, 0.08842893]
[12750, 354.26675, 344.00385, -10.262917, 0.11276986]
[12760, 280.58618, 271.93082, -8.655376, 0.08042102]
[12770, 299.9513, 291.67593, -8.275361, 0.08903015

[14150, 313.77164, 305.5071, -8.264516, 0.09499496]
[14160, 307.57993, 300.2613, -7.3186264, 0.09239534]
[14170, 292.0878, 284.02948, -8.058306, 0.08468332]
[14180, 301.8118, 294.61377, -7.198024, 0.09006767]
[14190, 308.43433, 300.76117, -7.6731563, 0.09102867]
[14200, 292.30176, 284.06815, -8.233622, 0.08539682]
[14210, 319.57144, 311.43686, -8.134576, 0.09763038]
[14220, 343.15756, 335.1046, -8.052945, 0.10828094]
[14230, 313.02295, 305.9768, -7.04615, 0.09385365]
[14240, 314.43405, 306.8227, -7.611361, 0.09672034]
[14250, 321.25778, 313.02304, -8.2347555, 0.09864584]
[14260, 301.81485, 294.3635, -7.451358, 0.0880031]
[14270, 316.53284, 309.29266, -7.2401743, 0.09694828]
[14280, 309.06973, 301.33044, -7.7392874, 0.092983]
[14290, 317.63806, 310.45407, -7.1840005, 0.09722697]
[14300, 317.94604, 310.71548, -7.230553, 0.097030625]
[14310, 317.89264, 310.06232, -7.830323, 0.098052524]
[14320, 320.2697, 313.28473, -6.9849725, 0.09581501]
[14330, 320.02402, 311.4947, -8.529341, 0.09723614

In [ ]:
real_images_split = batch_split(np_x, n_squares_side, n_channels)
stack_real_images_split = stack_batch_split(real_images_split)
print(stack_real_images_split.shape)

In [ ]:
batch_size_display = 10
hard = np.zeros(sp2.shape)
ind=1
scrambled_pieces = unflatten_batch(scrambled_pieces_split_np, n_channels)
joined_scrambled = join_batch_split(scrambled_pieces)
for i in range(batch_size_display):
    hard[i,0,:,:] =soft_to_hard(np.transpose(sgen[ind][i,0,:,:]))
unscrambled_images_hard =  np.matmul(hard[:batch_size_display,0,:,:], np.reshape(scrambled_pieces[:batch_size_display,:,:,:], [batch_size_display,n_squares_side **2 ,-1]))
unscrambled_images_soft =  ogen[ind]
unflatten_inf_hard = unflatten_batch(unscrambled_images_hard, n_channels)
joined_inf_hard = join_batch_split(unflatten_inf_hard)
unflatten_inf_soft = unflatten_batch(unscrambled_images_soft, n_channels)
joined_inf_soft = join_batch_split(unflatten_inf_soft)


In [ ]:
unscrambled_images_hard =  o2
unflatten_inf_hard = unflatten_batch(unscrambled_images_hard, n_channels)
joined_inf_hard = join_batch_split(unflatten_inf_hard)


In [ ]:
fig,ax=plt.subplots(10,3,figsize=(10,30))
#plt.imshow(joined_inf[0,:,:,0])
print(np_x.shape)
for i in range(10):
    
    
    ax[i,1].imshow(joined_inf_hard[i,:,:,0],'gray')
    ax[i,2].imshow(joined_inf_soft[i,:,:,0],'gray')
    ax[i,0].imshow(joined_scrambled[i,:,:,0],'gray')
    for k in range(3):
        ax[i,k].get_xaxis().set_visible(False)
        ax[i,k].get_yaxis().set_visible(False)

In [ ]:
print(np.max(ogen[1]))

In [ ]:
print(np.sum(ssp2[0,0,:,:], axis=1))
plt.figure(figsize=(20,20))
plt.imshow(ssp2[5,0,:,:])
print(ssp2[0,0,0,:])

In [ ]:
print(np_x)

In [ ]:

joined_inf_hard = join_batch_split(unflatten_batch(o2, n_channels))